#### This a a notebook of helper functions htat will be called by the feature engieering notebooks

In [4]:
from functools import reduce
import gensim
from gensim.test.utils import get_tmpfile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyspark
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel, IndexToString, StringIndexer, VectorAssembler
from pyspark.ml.linalg import Vectors, SparseVector, DenseVector, VectorUDT, _convert_to_vector
from pyspark.sql import Row, types, Window
from pyspark.sql.types import ArrayType, FloatType, StringType
from pyspark.sql import functions as F
import scipy
import tldextract

In [5]:
def create_featurized_df(data_path, new_col_names):
  '''
  This is a function that takes the path of a table stored on Databricks representing training
  data, a list of new column names, and creates a PySpark dataframe. This is necessary because
  the way column names are represented in our training data does not work with PySpark.
   
  Arguments: 
    data_path {string} -- String with the name of the table to be imported.
    new_col_names {list of strings} -- A list of column names to be assigned to the Dataframe.
    
  Returns:
    featurized {PySpark Dataframe} -- A PySpark Dataframe created from the data at data_path. 
      Columns are named in the order given in new_col_names.
  '''
  data = sqlContext.read.table(data_path)
  old_col_names = data.schema.names
  featurized = reduce(lambda data, idx: data.withColumnRenamed(old_col_names[idx], new_col_names[idx]), range(len(old_col_names)), data)
  return featurized

In [6]:
def prevalence_plot_nvalue(col, df, prevalence_pct=1, display_img=True, fig_size=(8, 4)):
  '''
  This is a function that takes a Pyspark dataframe and a column name and returns the 
  number of features that must be retained to explain a user-defined amount of variance 
  in the column.
  
  Arguments: 
    col {string} -- The name of the column under consideration.
    df {Pyspark Dataframe} -- The Pyspark Dataframe containing the column.
    prevalence_pct {float} -- Threshold indicating the percentage of values be retained. 
      Unique values are counted and sorted in descending order, then cumulatively summed 
      until this percentage of entries from the data are included.
    display_img {Boolean} -- Whether to display a plot of the cumulative sum. Default True.
    fig_size {tuple of ints} -- The size of the matplotlib plot to be displayed.
    
  Returns:
    n {int} -- The number of feature values to retain (of a descending sorted list of 
      feature prevalence.)
  '''
  col_series = df.groupBy(col).count().sort(F.desc("count"))
  total_sum = col_series.agg({"count":"sum"}).collect()[0][0]
  col_series = col_series.withColumn('variances', udf(lambda x: int(x)/total_sum)('count'))
  windowval = (Window.orderBy(F.col('count').desc()).rangeBetween(Window.unboundedPreceding, 0))
  col_series = col_series.withColumn('cum_sum', F.sum('variances').over(windowval))
  n = col_series.filter(F.col('cum_sum') <= prevalence_pct).count() 
  f, ax = plt.subplots(figsize=fig_size)
  ax.plot(col_series.select('cum_sum').collect())
  ax.vlines(n, 0, 1, linestyles='dashed')
  ax.annotate(
    '{}% Prevalence: N = {}'.format(prevalence_pct, n), 
    xy=(n, prevalence_pct), 
    xycoords='data', 
    xytext=(n * 3, prevalence_pct - .1), 
    arrowprops=dict(
      facecolor='black', 
      headwidth=10, 
      headlength=7, 
      width=1, 
      shrink=.05
    )
  )
  ax.set_xlabel('Number of Features')
  ax.set_ylabel('Prevalence Covered')
  f.suptitle('{} prevalence covered by top N features'.format(col))

  if display_img:
    display(f)
  return n

In [7]:
def get_n_values(n, col, df):
  '''
  A function to return a list of the n most prevalent values in a given column.
  Used after prevalence_plot_nvalue has returned the number of feature values to 
  retain.
  
  Arguments: 
    n {integer} -- The number of values to return.
    col {string} -- The name of the column under consideration.
    df {Pyspark Dataframe} -- The Pyspark Dataframe containing the column.
    
  Returns:
    final_values {list} -- A list of strings representing the values to retain
    from the column.
  '''
  col_series = df.groupBy(col).count().sort(F.desc("count"))
  vals = col_series.filter(~F.col(col).isin('nan','')).head(n)
  final_values = [x[0] for x in vals]
  return final_values

In [8]:
def new_col_values(val, final_values, fill=''):
  '''
  A UDF used to retain the values returned by prevalence_plot_nvalue and
  get_n_values and to replace others with empty strings.
  
  Arguments: 
    val {string} -- The string being considered for retention.
    final_values {list of strings} -- The list of value strings returned
      by get_n_values.
    
  Returns:
    result {string} -- A string representing the value.
  '''
  result = val if val in final_values else fill
  return result

In [9]:
def get_embedding_vectors(text, vectors):
  '''
  A UDF used to take strings and return their vector representation.
  
  Arguments: 
    text {string} -- The string being considered for retention.
    vectors {Gensim FastTextKeyedVectors} -- FastTextKeyedVectors
      trained by Gensim
    
  Returns:
    result {list of floats} -- List of floats representing a vector
      embedding (or 0.0s if text is an empty string).
  '''
  if text == '':
    return [float(0) for x in range(vectors.vector_size)]
  else:
    vector = vectors[text]
    return [float(x) for x in vector]

In [10]:
def binary_count_vectorize_column(data, input_col, output_col, create=True):
  '''
  A function to binary count vectorize a given column. This reduces lines of code compared to explicitly
  doing this for each relevant feature. It also saves the fit model so it can be called from the 
  notebook that does feature engineering for prediction.
     
  Arguments: 
    data {PySpark Dataframe} -- The dataframe having a binary count vectorize column added.
    input_col {string} -- The name of the column being binary count vectorized.
    output_col {string} -- The name of the new column.
    
  Returns:
    featurized {PySpark Dataframe} -- The provided dataframe with a new column added.
  '''
  temp_cv = CountVectorizer(inputCol=input_col, outputCol=output_col, binary=True)
  temp_model = temp_cv.fit(data)
  temp_model.write().overwrite().save('{}_model'.format(input_col))
  featurized = temp_model.transform(data)
  return featurized

In [11]:
def dense_to_sparse(array):
  '''
  A UDF to convert an array of strings into a sparse vector, which is the format required
  in our model pipeline. 
     
  Arguments: 
    array {array} -- An array of being converted into a sparse vector.
    
  Returns:
    A sparse vector.
  '''
  dv = DenseVector(array)
  return _convert_to_vector(scipy.sparse.csc_matrix(dv.toArray()).T)

In [12]:
def feature_engineering_training_pipeline(datestamp, data_tablename, cols=None, display=False, vector_len=100):
  '''
  A function to run the entire feature engineering for training pipeline. 
     
  Arguments: 
    datestamp {string} -- Today's date in MMDDYY format. Used to name the table.
    data_tablename {string} -- The name of the table of raw data used as input.
    cols {list of strings} -- a list of column names to be used if the schema changes.
    display {Boolean} -- Whether to display plots when prevalence_plot_nvalue is 
      called. Default false.
    vector_len {integer} -- Length of Fasttext vectors to create. Default 100.
    
  Returns:
    A string representing the name of the feature engineered dataset that has
      been saved.
  '''
  if cols is None:
    col_list = ['vulnerability_id',
    'assetName',
    'country',
    'status',
    'priority',
    'dataSource',
    'dnsName',
    'operatingSystem',
    'ipAdress',
    'findingTitle',
    'network',
    'port',
    'protocol',
    'netBios',
    'issue_name',
    'owner',
    'manager',
    'director',
    'vicePresident',
    'ml_owner', 
    'ml_predicted_owner', 
    'ownership_source', 
    'report_name'
               ]
  else:
    col_list = cols
  
  # Create a dataframe using the data with properly formatted names assigned.
  featurized = create_featurized_df(data_tablename, col_list)
  
  # Split the IP Address column into 4 separate columns.
  split_col = F.split(featurized['ipAdress'], '[.]')
  featurized = featurized.withColumn('ipAdress_1', split_col.getItem(0))
  featurized = featurized.withColumn('ipAdress_2', split_col.getItem(1))
  featurized = featurized.withColumn('ipAdress_3', split_col.getItem(2))
  featurized = featurized.withColumn('ipAdress_4', split_col.getItem(3))
  
  # Select just the columns that we are interested in.
  featurized = featurized.select(
  'ipAdress_1',
  'ipAdress_2',
  'ipAdress_3',
  'ipAdress_4',
  'country',
  'dataSource',
  'dnsName',
  'operatingSystem',
  'network',
  'port',
  'protocol',
  'issue_name',
  'owner',
  'vulnerability_id'
  )
  
  # Run prevalence plot feature selection steps for issue_name, port, operatingSystem
  # issue_name
  feats = prevalence_plot_nvalue("issue_name", featurized, display_img=display)
  final_values = get_n_values(feats, "issue_name", featurized)
  featurized = featurized.withColumn('issue_name_new', udf(lambda x: new_col_values(x, final_values))(F.col('issue_name')))
  
  # port
  feats = prevalence_plot_nvalue('port', featurized, display_img=display)
  final_values = get_n_values(feats, 'port', featurized)
  featurized = featurized.withColumn('port_new', udf(lambda x: new_col_values(x, final_values))(F.col("port")))
  
  # operatingSystem
  # We're using the first two words describing the operating system.
  featurized = featurized.withColumn('operatingSystem_int', udf(lambda x: ' '.join(str(x).split()[:2]))(F.col("operatingSystem")))
  
  feats = prevalence_plot_nvalue('operatingSystem_int', featurized, display_img=display)
  final_values = get_n_values(feats, 'operatingSystem_int', featurized)
  featurized = featurized.withColumn('operatingSystem_new', udf(lambda x: new_col_values(x, final_values))(F.col("operatingSystem_int")))
  
  # owner (add a new column that labels the 1% of least common owners all as a class 'unknown')
  feats = prevalence_plot_nvalue('owner', featurized, prevalence_pct=.99, display_img=False)
  final_values = get_n_values(feats, 'owner', featurized)
  featurized = featurized.withColumn('owner_new', udf(lambda x: new_col_values(x, final_values, fill='unknown'))(F.col("owner")))
  
  # Split dnsName into component parts, then encode them using Fasttext
  featurized = featurized.withColumn('dns_domain', udf(lambda x: tldextract.extract(str(x)).domain)(F.col('dnsName')))
  featurized = featurized.withColumn('dns_suffix', udf(lambda x: tldextract.extract(str(x)).suffix)(F.col('dnsName')))
  featurized = featurized.withColumn('dns_subdomain', udf(lambda x: tldextract.extract(str(x)).subdomain)(F.col('dnsName')))
  
  # Identify rows where `dns_subdomain` is missing.
  featurized = featurized.withColumn('dns_subdomain_isnull', udf(lambda x: 1 if x == '' else 0)(F.col('dns_subdomain')))
  
  # Create a list of all subdomains to use as our fasttext vocabulary.
  dns_subdomain_list = [[row.dns_subdomain] for row in featurized.select('dns_subdomain').where(featurized.dns_subdomain != '').collect()]
  
  # Create fasttext word embeddings.
  vector_size = vector_len
  model = gensim.models.FastText(size=vector_size)
  model.build_vocab(sentences=dns_subdomain_list)
  model.train(sentences=dns_subdomain_list, total_examples=len(dns_subdomain_list), epochs=10)
  dns_vectors = model.wv
  
  # Save out the KeyedVectors
  fname = get_tmpfile('/dbfs/fasttext_dns_vectors_{}.kv'.format(vector_size))
  dns_vectors.save(fname)
  
  # Retrieve trained vectors of subdomains and create a new column.
  featurized = featurized.withColumn('dns_subdomain_vector', udf(lambda x: get_embedding_vectors(x, dns_vectors), ArrayType(FloatType()))(F.col('dns_subdomain')))
  
  # One Hot Encode the other features.
  featurized = featurized.select(
    'owner', 
    'vulnerability_id', 
    'ipAdress_1', 
    'ipAdress_2', 
    'ipAdress_3', 
    'ipAdress_4', 
    'issue_name_new', 
    'port_new', 
    'operatingSystem_new', 
    'country', 
    'protocol', 
    'dataSource', 
    'network', 
    'dns_subdomain_vector', 
    'dns_domain', 
    'dns_suffix', 
    'dns_subdomain_isnull'
  )
  
  # These are the columns that actually need to be vecorized - dns_subdomain_vector is already a vector, 
  # owner is the target variable, and vulnerability_id is supposed to just be passed through.
  vectorize_cols = list(set(featurized.columns) - set(['dns_subdomain_vector', 'owner', 'vulnerability_id']))
  
  # In order to get count vectorized, every non-array feature needs to be turned into one.
  for col in vectorize_cols:
    featurized = featurized.withColumn(col, F.array(featurized[col]))
    featurized = binary_count_vectorize_column(featurized, col, col + '_vectors')
    
  # This step converts the dns_subdomain_vector array to a sparse array udt so it'll work later with our model pipeline.
  featurized = featurized.withColumn('dns_subdomain_vector', udf(dense_to_sparse, VectorUDT())(F.col('dns_subdomain_vector')))
  
  # Select the relevant columns 
  featurized = featurized.select(
    'dns_subdomain_vector', 
    'ipAdress_1_vectors', 
    'ipAdress_2_vectors',   
    'ipAdress_3_vectors',   
    'ipAdress_4_vectors',   
    'issue_name_new_vectors',   
    'port_new_vectors',   
    'operatingSystem_new_vectors',   
    'country_vectors',   
    'protocol_vectors',  
    'dataSource_vectors',   
    'network_vectors',   
    'dns_domain_vectors',   
    'dns_suffix_vectors',  
    'dns_subdomain_isnull_vectors',  
    'owner',
    'vulnerability_id'
  )
  
  # Write out the table.
  output_table_name = 'ads_training_embedded_dns_len_{}_{}'.format(vector_size, datestamp)
  featurized.write.saveAsTable(output_table_name)
  
  return output_table_name